In [ ]:
import cv2
import numpy as np
import time

video_path = "../video/rilevamento-intrusioni-video.wm"
cap = cv2.VideoCapture(video_path)

assert cap.isOpened(), "Not opened!"

fps = int(cap.get(cv2.CAP_PROP_FPS))
length = cap.get(cv2.CAP_PROP_FRAME_COUNT) / fps

print(f"[I] Video FPS: {fps}")
print(f"[I] Video Length: {length}")

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"[I] Video Frame Width: {width}")
print(f"[I] Video Frame height: {height}")

In [2]:
# MOG2 Background Subtractor
mog2 = cv2.createBackgroundSubtractorMOG2(history=fps * 10, varThreshold=2, detectShadows=False)
mog2.setNMixtures(1)

In [3]:
# """
#     Performed a sorted insertion of pv into the frame buffer in the pi row.
#     pv: pixel value
#     pi: pixel index
#     frame_buffer (global)
# """
# def sortAndInsert(pv, pi, ):
#     global frame_buffer
#     insertion_idx = np.searchsorted(frame_buffer[pi], pv)
#     if insertion_idx == len(frame_buffer[pi]):
#         insertion_idx =- 1

#     frame_buffer[pi, insertion_idx] = pv

In [ ]:
MAX_HISTORY = 5 * fps
SKIP_FRAMES = fps

# 320x240 rows, 60 columns
# frame_buffer = np.empty((width * height,MAX_HISTORY), dtype=np.uint8)

start = time.time()
frames = []
initialized = False
frame_count = 0
skip_count = 0
temporalMeadianBackground = None

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret or frame is None:
        cap.release()
        print("Released Video Resource")
        break
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_count += 1
    skip_count += 1

    if initialized:
        # print(f"[I] Skipping Frames... {skip_count}")
        if skip_count > SKIP_FRAMES:
            skip_count = 0
            frames.pop(0)
            frames.append(frame)
            # print(f"[I] Added Image...")

            temporalMeadianBackground = np.median(frames, axis=0).astype(dtype=np.uint8)
            cv2.imshow(f"temporalMeadianBackground-frame-{frame_count}", temporalMeadianBackground)
            cmd = cv2.waitKey(0)
            cv2.destroyWindow(f"temporalMeadianBackground-frame-{frame_count}")
            if cmd == ord("q"):
                break
            if cmd == ord("n"):
                continue
    
    if not initialized and len(frames) < MAX_HISTORY:
        # print(f"[I] Learning...")
        frames.append(frame)
        
    elif not initialized and len(frames) == MAX_HISTORY:
        # print(f"Compute First Estimate")
        # compute the first background estimation
        initialized = True    
        temporalMeadianBackground = np.median(frames, axis=0).astype(dtype=np.uint8)
        cv2.imshow("temporalMeadianBackground-1", temporalMeadianBackground)
        cmd = cv2.waitKey(0)
        cv2.destroyWindow("temporalMeadianBackground-1") 
        if cmd == ord("q"):
            break
        if cmd == ord("n"):
            continue
    

end = time.time()

print(f"[I] Elapsed time: {end - start} s")

cap.release()
cv2.destroyAllWindows()
